# Contract Analysis with Azure SQL - Local Testing

This notebook breaks down the Azure Function logic for step-by-step local testing.

## Prerequisites
- Azure CLI logged in (`az login`)
- ODBC Driver 18 for SQL Server installed
- Azure SQL Database created with tables (run `create_tables.sql`)
- Managed identity or your user granted access to the database
- PDF contract files in `../contracts/` folder

## Step 1: Install Dependencies

In [5]:
# Uncomment to install dependencies
# %pip install azure-identity requests pyodbc

## Step 2: Import Libraries

In [6]:
import os
import json
import time
import struct
import requests
import pyodbc
from pathlib import Path
from azure.identity import DefaultAzureCredential

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Step 3: Configuration

Set your Azure resource details here.

In [ ]:
# Content Understanding configuration
# CONTENT_UNDERSTANDING_ENDPOINT = "https://agentic-contracts-proj-resource.cognitiveservices.azure.com"  # e.g., "https://your-resource.cognitiveservices.azure.com"
# https://agentic-contracts-proj-resource.services.ai.azure.com
CONTENT_UNDERSTANDING_ENDPOINT = "https://agentic-contracts-proj-resource.services.ai.azure.com"

CONTENT_UNDERSTANDING_API_VERSION = "2025-11-01"
CONTENT_UNDERSTANDING_ANALYZER_ID = (
    "projectAnalyzer_1769494397437_218"  # e.g., "projectAnalyzer_123456_789"
)
CONTENT_UNDERSTANDING_ANALYZER_ID= "prebuilt-contract"

# Azure SQL configuration
SQL_SERVER = "contract-sql-db"  # e.g., "my-sql-server" (without .database.windows.net)
SQL_DATABASE = "contracts-db"  # e.g., "contracts-db"

# Validate configuration
config_valid = all([
    CONTENT_UNDERSTANDING_ENDPOINT,
    CONTENT_UNDERSTANDING_ANALYZER_ID,
    SQL_SERVER,
    SQL_DATABASE
])

if config_valid:
    print("✓ Configuration set")
    print(f"  Content Understanding: {CONTENT_UNDERSTANDING_ENDPOINT}")
    print(f"  Analyzer ID: {CONTENT_UNDERSTANDING_ANALYZER_ID}")
    print(f"  SQL Server: {SQL_SERVER}.database.windows.net")
    print(f"  Database: {SQL_DATABASE}")
else:
    print("❌ Please fill in all configuration values above")

✓ Configuration set
  Content Understanding: https://agentic-contracts-proj-resource.cognitiveservices.azure.com
  Analyzer ID: projectAnalyzer_1769494397437_218
  SQL Server: contract-sql-db.database.windows.net
  Database: contracts-db


## Step 4: Authenticate with Azure

Uses `DefaultAzureCredential` which works with:
- Azure CLI (`az login`)
- Managed Identity (when deployed)
- VS Code Azure extension

In [8]:
# Authenticate and get tokens
credential = DefaultAzureCredential()

# Token for Content Understanding
cu_token = credential.get_token("https://cognitiveservices.azure.com/.default")
print("✓ Content Understanding token acquired")

# Token for Azure SQL
sql_token = credential.get_token("https://database.windows.net/.default")
print("✓ Azure SQL token acquired")

✓ Content Understanding token acquired
✓ Azure SQL token acquired


## Step 5 : Create tables using create_tables.sql

## Step 6: Test Azure SQL Connection

In [9]:
def get_sql_connection(server: str, database: str, token: str):
    """Create Azure SQL connection using Azure AD token."""
    # Encode token for pyodbc
    token_bytes = token.encode("UTF-16-LE")
    token_struct = struct.pack(f"<I{len(token_bytes)}s", len(token_bytes), token_bytes)
    
    conn_str = (
        f"Driver={{ODBC Driver 18 for SQL Server}};"
        f"Server={server}.database.windows.net,1433;"
        f"Database={database};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=no;"
    )
    
    return pyodbc.connect(conn_str, attrs_before={1256: token_struct})

# Test connection
try:
    conn = get_sql_connection(SQL_SERVER, SQL_DATABASE, sql_token.token)
    cursor = conn.cursor()
    cursor.execute("SELECT @@VERSION")
    version = cursor.fetchone()[0]
    print("✓ Connected to Azure SQL")
    print(f"  {version[:50]}...")
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")

✓ Connected to Azure SQL
  Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Jan  2 2...


## Step 7: Verify Tables Exist

In [10]:
conn = get_sql_connection(SQL_SERVER, SQL_DATABASE, sql_token.token)
cursor = conn.cursor()

# Check for tables
cursor.execute("""
    SELECT TABLE_NAME 
    FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_TYPE = 'BASE TABLE'
    ORDER BY TABLE_NAME
""")

tables = [row[0] for row in cursor.fetchall()]
required_tables = ['contracts', 'parties', 'clauses']

print("Tables in database:")
for table in tables:
    status = "✓" if table in required_tables else " "
    print(f"  {status} {table}")

missing = set(required_tables) - set(tables)
if missing:
    print(f"\n❌ Missing tables: {missing}")
    print("   Run create_tables.sql first!")
else:
    print("\n✓ All required tables exist")

cursor.close()
conn.close()

Tables in database:
  ✓ clauses
  ✓ contracts
  ✓ parties

✓ All required tables exist


## Step 8: Load Contract Files

In [11]:
# Get PDF files from contracts folder
contracts_dir = Path("./contracts")
contract_files = list(contracts_dir.glob("*.pdf"))

print(f"Found {len(contract_files)} contract files:")
for file in contract_files:
    size_kb = file.stat().st_size / 1024
    print(f"  - {file.name} ({size_kb:.1f} KB)")

Found 1 contract files:
  - contract_sample_1.pdf (2.0 KB)


## Step 9: Define Field Extraction Helpers

These functions extract structured data from Content Understanding responses.

In [12]:
def extract_field_value(field_data):
    """Extract value from Content Understanding field structure."""
    if not field_data:
        return None

    field_type = field_data.get("type")

    if field_type == "array":
        return [extract_field_value(item) for item in field_data.get("valueArray", [])]
    elif field_type == "object":
        return {k: extract_field_value(v) for k, v in field_data.get("valueObject", {}).items()}
    elif field_type == "string":
        return field_data.get("valueString")
    elif field_type == "number":
        return field_data.get("valueNumber")
    elif field_type == "date":
        return field_data.get("valueDate")
    else:
        return field_data.get("content", field_data.get("valueString"))


def extract_array_field(field_data) -> list:
    """Extract array field values."""
    if not field_data:
        return []
    return [
        v for item in field_data.get("valueArray", [])
        if (v := extract_field_value(item)) is not None
    ]


def extract_parties(fields) -> list:
    """Extract party information from contract fields."""
    parties = []
    for party_item in fields.get("Parties", {}).get("valueArray", []):
        if party_item.get("type") == "object":
            party_obj = party_item.get("valueObject", {})
            party = {
                "name": extract_field_value(party_obj.get("Name")),
                "address": extract_field_value(party_obj.get("Address")),
                "reference_name": extract_field_value(party_obj.get("ReferenceName")),
                "clause": extract_field_value(party_obj.get("Clause")),
            }
            if party["name"]:
                parties.append({k: v for k, v in party.items() if v is not None})
    return parties


def extract_dates(fields) -> dict:
    """Extract date information from contract fields."""
    date_fields = ["ExecutionDate", "EffectiveDate", "ExpirationDate", "RenewalDate"]
    return {
        field: value
        for field in date_fields
        if (value := extract_field_value(fields.get(field)))
    }


def extract_clauses(fields) -> list:
    """Extract clause information from contract fields."""
    clauses = []
    for clause_item in fields.get("Clauses", {}).get("valueArray", []):
        if clause_item.get("type") == "object":
            clause_obj = clause_item.get("valueObject", {})
            clause = {
                "type": extract_field_value(clause_obj.get("clauseType")),
                "title": extract_field_value(clause_obj.get("clauseTitle")),
                "text": extract_field_value(clause_obj.get("clauseText")),
            }
            if clause["title"] or clause["text"]:
                clauses.append({k: v for k, v in clause.items() if v is not None})
    return clauses


print("✓ Field extraction helpers defined")

✓ Field extraction helpers defined


## Step 10: Define Contract Analysis Function

In [13]:
def analyze_contract_document(file_content: bytes, filename: str, endpoint: str, token: str) -> dict:
    """Analyze a contract document using Azure Content Understanding."""
    print(f"Analyzing: {filename}")

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/pdf",
    }

    # Submit analysis request
    print("  Submitting to Content Understanding...")
    response = requests.post(endpoint, headers=headers, data=file_content)

    if response.status_code != 202:
        print(f"  ❌ API error: {response.status_code}")
        return {"filename": filename, "error": f"API returned {response.status_code}: {response.text}"}

    operation_location = response.headers.get("Operation-Location")
    if not operation_location:
        return {"filename": filename, "error": "No Operation-Location header"}

    # Poll for results
    print("  Waiting for analysis...")
    poll_headers = {"Authorization": f"Bearer {token}"}

    for attempt in range(60):
        time.sleep(2)
        result_response = requests.get(operation_location, headers=poll_headers)

        if result_response.status_code == 200:
            result_data = result_response.json()
            status = result_data.get("status")
            
            if attempt % 5 == 0:
                print(f"  Status: {status}")

            if status == "Succeeded":
                contents = result_data.get("result", {}).get("contents", [])
                if contents:
                    fields = contents[0].get("fields", {})
                    print("  ✓ Analysis complete")
                    return {
                        "filename": filename,
                        "title": extract_field_value(fields.get("Title")),
                        "parties": extract_parties(fields),
                        "dates": extract_dates(fields),
                        "duration": extract_field_value(fields.get("ContractDuration")),
                        "jurisdictions": extract_array_field(fields.get("Jurisdictions")),
                        "clauses": extract_clauses(fields),
                        "raw_fields": fields,
                        "markdown": contents[0].get("markdown", ""),
                    }
                return {"filename": filename, "raw_result": result_data}

            elif status in ["Failed", "Canceled"]:
                print(f"  ❌ Analysis {status.lower()}")
                return {"filename": filename, "error": f"Analysis {status.lower()}"}

    return {"filename": filename, "error": "Timeout waiting for results"}


print("✓ Contract analysis function defined")

✓ Contract analysis function defined


## Step 11: Analyze a Single Contract (Test)

In [14]:
# Test with the first contract file
if contract_files:
    test_file = contract_files[0]
    print(f"Testing with: {test_file.name}\n")
    
    # Build endpoint
    endpoint = f"{CONTENT_UNDERSTANDING_ENDPOINT}/contentunderstanding/analyzers/{CONTENT_UNDERSTANDING_ANALYZER_ID}:analyzeBinary?api-version={CONTENT_UNDERSTANDING_API_VERSION}"
    
    # Read file
    with open(test_file, "rb") as f:
        file_content = f.read()
    
    # Analyze
    result = analyze_contract_document(
        file_content=file_content,
        filename=test_file.name,
        endpoint=endpoint,
        token=cu_token.token
    )
    
    # Store result for later use
    test_result = result
else:
    print("❌ No contract files found in ../contracts/")

Testing with: contract_sample_1.pdf

Analyzing: contract_sample_1.pdf
  Submitting to Content Understanding...
  ❌ API error: 404


## Step 12: View Extracted Data

In [15]:
# Display the extracted data
if 'test_result' in dir() and 'error' not in test_result:
    print(f"📄 Contract: {test_result['filename']}")
    print(f"{'='*60}")
    
    if test_result.get('title'):
        print(f"\n📌 Title: {test_result['title']}")
    
    # Parties
    print("\n👥 Parties:")
    for party in test_result.get('parties', []):
        print(f"  • {party.get('name', 'Unknown')}")
        if party.get('address'):
            print(f"    Address: {party['address']}")
    
    # Dates
    print("\n📅 Dates:")
    for date_type, date_value in test_result.get('dates', {}).items():
        print(f"  {date_type}: {date_value}")
    
    # Duration
    if test_result.get('duration'):
        print(f"\n⏱️ Duration: {test_result['duration']}")
    
    # Jurisdictions
    if test_result.get('jurisdictions'):
        print(f"\n🌍 Jurisdictions: {', '.join(test_result['jurisdictions'])}")
    
    # Clauses summary
    print(f"\n📝 Clauses: {len(test_result.get('clauses', []))} found")
    for i, clause in enumerate(test_result.get('clauses', [])[:5], 1):
        print(f"  [{i}] {clause.get('title', 'Untitled')}")
    if len(test_result.get('clauses', [])) > 5:
        print(f"  ... and {len(test_result['clauses']) - 5} more")
else:
    print("❌ No successful analysis result to display")
    if 'test_result' in dir():
        print(f"Error: {test_result.get('error', 'Unknown')}")

❌ No successful analysis result to display
Error: API returned 404: {"error":{"code":"NotFound","message":"Resource not found.","innererror":{"code":"ModelNotFound","message":"The requested model was not found."}}}


## Step 13: Define Database Storage Function

In [16]:
def store_in_database(result: dict, server: str, database: str, token: str) -> dict:
    """Store the analysis results in Azure SQL Database."""
    conn = get_sql_connection(server, database, token)
    cursor = conn.cursor()

    # Insert contract
    cursor.execute(
        """
        INSERT INTO contracts (filename, title, duration, jurisdictions, dates, markdown, raw_fields)
        OUTPUT INSERTED.id
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (
            result.get("filename"),
            result.get("title"),
            result.get("duration"),
            json.dumps(result.get("jurisdictions", [])),
            json.dumps(result.get("dates", {})),
            result.get("markdown"),
            json.dumps(result.get("raw_fields", {})),
        ),
    )
    contract_id = cursor.fetchone()[0]
    print(f"  ✓ Contract inserted (id={contract_id})")

    # Insert parties
    party_ids = []
    for party in result.get("parties", []):
        cursor.execute(
            """
            INSERT INTO parties (contract_id, name, address, reference_name, clause)
            OUTPUT INSERTED.id
            VALUES (?, ?, ?, ?, ?)
            """,
            (
                contract_id,
                party.get("name"),
                party.get("address"),
                party.get("reference_name"),
                party.get("clause"),
            ),
        )
        party_ids.append(cursor.fetchone()[0])
    print(f"  ✓ {len(party_ids)} parties inserted")

    # Insert clauses
    clause_ids = []
    for clause in result.get("clauses", []):
        cursor.execute(
            """
            INSERT INTO clauses (contract_id, clause_type, title, text)
            OUTPUT INSERTED.id
            VALUES (?, ?, ?, ?)
            """,
            (
                contract_id,
                clause.get("type"),
                clause.get("title"),
                clause.get("text"),
            ),
        )
        clause_ids.append(cursor.fetchone()[0])
    print(f"  ✓ {len(clause_ids)} clauses inserted")

    conn.commit()
    cursor.close()
    conn.close()

    return {
        "contract_id": contract_id,
        "party_ids": party_ids,
        "clause_ids": clause_ids,
    }


print("✓ Database storage function defined")

✓ Database storage function defined


## Step 14: Store Test Result in Database

In [17]:
# Store the test result in Azure SQL
if 'test_result' in dir() and 'error' not in test_result:
    print(f"Storing: {test_result['filename']}")
    
    # Get fresh token (tokens expire after ~1 hour)
    sql_token = credential.get_token("https://database.windows.net/.default")
    
    db_result = store_in_database(
        result=test_result,
        server=SQL_SERVER,
        database=SQL_DATABASE,
        token=sql_token.token
    )
    
    print(f"\n✓ Stored successfully!")
    print(f"  Contract ID: {db_result['contract_id']}")
    print(f"  Party IDs: {db_result['party_ids']}")
    print(f"  Clause IDs: {db_result['clause_ids']}")
else:
    print("❌ No successful analysis result to store")

❌ No successful analysis result to store


## Step 15: Query Stored Data

In [18]:
import pandas as pd

# Query the stored data
sql_token = credential.get_token("https://database.windows.net/.default")
conn = get_sql_connection(SQL_SERVER, SQL_DATABASE, sql_token.token)

# Contracts
print("📄 CONTRACTS")
contracts_df = pd.read_sql("SELECT id, filename, title, duration, created_at FROM contracts ORDER BY created_at DESC", conn)
display(contracts_df)

# Parties
print("\n👥 PARTIES")
parties_df = pd.read_sql("SELECT id, contract_id, name, address FROM parties", conn)
display(parties_df)

# Clauses
print("\n📝 CLAUSES")
clauses_df = pd.read_sql("SELECT id, contract_id, clause_type, title, LEFT(text, 100) as text_preview FROM clauses", conn)
display(clauses_df)

conn.close()

📄 CONTRACTS


/tmp/ipykernel_1156/3689242772.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  contracts_df = pd.read_sql("SELECT id, filename, title, duration, created_at FROM contracts ORDER BY created_at DESC", conn)


,id,filename,title,duration,created_at
0,1,contract_sample_1.pdf,SUPPLY VENDOR AGREEMENT,36 months,2026-01-27 07:03:11.820



👥 PARTIES


/tmp/ipykernel_1156/3689242772.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  parties_df = pd.read_sql("SELECT id, contract_id, name, address FROM parties", conn)


,id,contract_id,name,address
0,1,1,"ACME Medical Supplies, Inc.",None
1,2,1,Riverbend Health System,None



📝 CLAUSES


/tmp/ipykernel_1156/3689242772.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  clauses_df = pd.read_sql("SELECT id, contract_id, clause_type, title, LEFT(text, 100) as text_preview FROM clauses", conn)


,id,contract_id,clause_type,title,text_preview


## Step 17: Create Function App on portal.azure.com
### Follow azure_function_sql/README.md to upload the function to Azure